# Pairwise Sequence Alignment - Elaborato per il corso Ricerca Operativa

<img src="assets/dna2.png" alt= “” width="300" >


### Descrizione del problema
L'allineamento di sequenza a coppie è un problema fondamentale nell'ambito della bioinformatica e dell'analisi delle sequenze genetiche. L'obiettivo è confrontare due sequenze di DNA, RNA o proteine per identificare regioni di somiglianza o di conservazione funzionale.

Le sequenze biologiche possono presentare **variazioni e mutazioni** nel corso dell'evoluzione, rendendo necessario l'allineamento per rivelare le corrispondenze tra i loro elementi costitutivi. Se due sequenze condividono un antenato comune, le mancate corrispondenze (*mismatch*) possono essere interpretate come mutazioni puntiformi e i buchi (*gap*) come indel (mutazioni di inserimento o cancellazione).

### Allineamenti validi 
Un allineamento tra due stringhe viene ottenuto aggiungendo simboli di gap ("-") a ciascuna stringa in modo che le stringhe risultanti abbiano la stessa lunghezza. Di conseguenza, i simboli in posizioni corrispondenti possono essere sovrapposti per formare l'allineamento. È importante che i simboli di gap non vengano allineati tra di loro, in modo che rappresentino l'inserimento o la cancellazione di un simbolo nella stringa opposta.

### Il Modello
Di seguito viene presentato il modello formulato per descrivere il problema dell'allineamento di sequenze a coppie. È possibile impostare i parametri di penalità per i mismatch e i gap, così come il valore di ricompensa per le corrispondenze.

La funzione obiettivo cerca di massimizzare il numero di corrispondenze e minimizzare il numero di mismatch e di gap, inclusi i gap finali. Il costo ottimo corrisponde all'opposto del punteggio dell'allineamento.

In [10]:
%%writefile ampl_files/SequenceAlignment.mod

param n;                               # length of sequence s1
param m;                               # length of sequence s2
param mis_p;                           # mismatch penality
param gap_p;                           # gap penality
param r;                               # match reward
param a{i in 1..n, j in 1..m} binary;  # matrix that indicates whether s1[i] is equal to s2[j]

var x{i in 1..n, j in 1..m} binary;    # matrix that indicates whether s1[i] and s2[j] are aligned
var y{i in 1..n} binary;               # array that indicates whether s1[i] is aligned with a gap in s2
var z{j in 1..m} binary;               # array that indicates whether s2[j] is aligned with a gap in s1
var v;

# a symbol in a sequence can't be aligned with both another symbol and with a gap at the same time.
subject to maxAlignmentsS1{i in 1..n}: 
	(sum{j in 1..m} x[i,j]) + y[i] <= 1;
subject to maxAlignmentsS2{j in 1..m}: 
	(sum{i in 1..n} x[i,j]) + z[j] <= 1;
    
# if s1[i] is aligned with s2[j], previous symbols in s1 can't be aligned with following symbols 
# in s2, and vice versa.
subject to alignmentOrder1{i in 1..n, j in 1..m, k in 1..i-1, h in j+1..m}:
    x[i,j] + x[k,h] <= 1;
subject to alignmentOrder2{i in 1..n, j in 1..m, k in i+1..n, h in 1..j-1}:
    x[i,j] + x[k,h] <= 1;
    
# two symbols to be aligned must be the same.
subject to sameSymbol{i in 1..n, j in 1..m}: 
	x[i,j] <= a[i,j];
    
# if two symbols are aligned but have different positions in their sequence, there must be 
# enough gaps in previous positions of both the sequences.
subject to previousGaps{i in 1..n, j in 1..m}:
	x[i,j] * ( ( sum{k in 1..i} y[k] ) - ( sum{h in 1..j} z[h] ) ) == x[i,j] * (i - j);

# to know the number of final gaps in the shortest sequence, we need to calculate the absolute value 
# of the difference between the two aligned sequences. Since the absolute value is not a linear function,
# we add the following two constraints to set the variable v to be the absolute value we need.
subject to absoluteValueA:
	-v <= (n + sum{j in 1..m} z[j]) - (m + sum{i in 1..n} y[i]);
subject to absoluteValueB:
	(n + sum{j in 1..m} z[j]) - (m + sum{i in 1..n} y[i]) <= v;
    
# with this objective function we try to maximize the number of matches and 
# at the same time minimize the number of gaps and mismatches.
minimize obj: 
	( sum{j in 1..m} sum{i in 1..n} (x[i,j]) * r) + #matches
	( sum{i in 1..n} y[i] * gap_p )               + #gaps
	( sum{j in 1..m} z[j] * gap_p )               + 
	( sum{i in 1..n} (1 - ( ( sum{j in 1..m} x[i,j] ) + y[i] ) ) * mis_p ) +  #mismatches
	( v * gap_p ); #ending gaps
	


Overwriting ampl_files/SequenceAlignment.mod


### Il Codice

Importiamo il modulo amplpy

In [11]:
from amplpy import AMPL, add_to_path
add_to_path(r"C:\ampl") # path to the ampl installation folder

La funzione di utilità ```writeData``` crea un file di dati nel formato richiesto per l'utilizzo del software AMPL, a partire dalle due sequenze da allineare fornite come input. 

In [12]:
def writeData(s1, s2, filename = "ampl_files/input.dat"):
    
    file = open(filename, "w")
    file.writelines([
            f"param n:={len(s1)};\n",
            f"param m:={len(s2)};\n",
            "param gap_p:=1;\n", # gap penality
            "param mis_p:=1;\n", # mismatch penality
            "param r:=-1;\n",    # match reward
            "param a :=\n"
        ])
    
    a = []
    for c1 in s1:
      row = []
      for c2 in s2:
        row.append(1 if c1 == c2 else 0)
      a.append(row)
    
    for i in range(len(a)):
      for j in range(len(a[i])):
        file.write(f"{i+1} {j+1} {a[i][j]} \n")
    
    file.write(";\n")
    file.close()

La funzione di utilità ```addGaps``` costruisce le stringhe con i gap a partire da due array, *y* e *z*, che rappresentano le corrispondenze a gap nelle due sequenze.

In [13]:
def addGaps(a, b, y, z):
    
    gaps1 = []
    for (i,v) in y:
        if round(v)==1:
            gaps1.append(int(i))
    gaps2 = []
    for (i,v) in z:
        if round(v)==1:
            gaps2.append(int(i))
            
    for i in range(1,max(len(a), len(b))+1):
        if i in gaps1:
            b = b[:i-1] + "-" + b[i-1:]
            gaps1.remove(i)
            gaps2 = [k+1 for k in gaps2]
        if i in gaps2:
            a = a[:i-1] + "-" + a[i-1:]
            gaps2.remove(i)
            gaps1 = [k+1 for k in gaps1]
    
    return " ".join(a), " ".join(b)

La funzione ```solveAlignment``` risolve il problema dell'allineamento di sequenze a coppie utilizzando il software AMPL e restituisce il punteggio ottimo e le sequenze allineate con i gap aggiunti. La funzione accetta due sequenze *a* e *b* come input.

L'opzione ```solver_msg 0``` e il reindirizzamento di solve in ```ampl_files/out.txt``` servono a evitare la visualizzazione di output durante l'esecuzione. Tuttavia, se si desidera visualizzare il numero di iterazioni fatte e il valore del costo ottimo, è possibile rimuovere queste opzioni.

In [14]:
def solveAlignment(a,b):
    writeData(a, b)
    
    ampl = AMPL()
    
    ampl.eval(r''' 
            reset;
            model ampl_files/SequenceAlignment.mod;
            data ampl_files/input.dat;
            option solver cbc;
            option solver_msg 0; 
            solve >ampl_files/out.txt;
            ''')
    
    score = -round(ampl.get_value("obj"))
    
    y = ampl.getVariable("y").getValues()        
    z = ampl.getVariable("z").getValues()
    
    return score, addGaps(a, b, y, z)

### Esempio
Di seguito è mostrato un esempio di utilizzo del modello. 

Prese due sequenze *s1* e *s2*, usiamo la funzione ```solveAlignment``` per calcolare l'allineamento ottimo e il suo punteggio.

In [15]:
s1 = "GCCAGTTGG"
s2 = "GTCGTTAAACGCA"

score, result = solveAlignment(s1,s2)
s1_with_gaps, s2_with_gaps = result

Visualizziamo a schermo il risultato.

In [16]:
print("Optimal alignment:")
print(s1_with_gaps)
print(s2_with_gaps)
print(f"Score = {score}")

Optimal alignment:
G C C A G T T - - G - G
G T C - G T T A A A C G C A
Score = -2


### Test
La funzione ```getScore``` utilizza il modulo [minineedle](https://github.com/scastlara/minineedle), che implementa l'algoritmo di Needleman-Wunsch.\
L'algoritmo di Needleman-Wunsch è un algoritmo di programmazione dinamica utilizzato in bioinformatica per risolvere l'allineamento di sequenza.

In [17]:
from minineedle import needle, core
import numpy as np

def getScore(a,b):
    alignment: needle.NeedlemanWunsch[str] = needle.NeedlemanWunsch(a, b)
    alignment.change_matrix(core.ScoreMatrix(match=1, miss=-1, gap=-1))
    alignment.align()
    return alignment.get_score()


Per confrontare i risultati del modello proposto con il metodo di Needleman-Wunsch, eseguiamo una valutazione su un set di 10 coppie di sequenze casuali.

In [18]:
n_tests = 10

for i in range(n_tests):
    a = "".join(np.random.choice(['A', 'T', 'G', 'C'], np.random.randint(3,15)))
    b = "".join(np.random.choice(['A', 'T', 'G', 'C'], np.random.randint(3,15)))
    score = solveAlignment(a,b)[0]
    correct = getScore(a,b)
    assert score==correct, "Alignment test failed"
    print(f"\r{i+1}/{n_tests}",end="")
    
print("\nAll tests completed correctly!")


10/10
All tests completed correctly!
